### Convolutional Neural Network

합성곱 신경망(Convolutional Neural network, CNN)은 뇌의 시각 피질이 물체를 인식할 때 동작하는 방식에서 영감을 얻은 모델이며, 각 층별로 저수준 특성을 연결하여 고수준 특성을 만듦으로써 특성 계층을 구성한다

- 국부 수용장(local receptive field): 위와 같은 국부적인 픽셀 패치
- 희소 연결: 특성 맵에 있는 하나의 원소는 작은 픽셀 패치 하나에만 연결됨
- 파라미터 공유: 동일한 가중치가 입력 이미지의 모든 패치에 사용됨

- CNN은 합성곱(conv)층과 풀링(Pooling, P)층이라고 하는 서브샘플링 층으로 이루어져 있는데, 마지막에는 하나 이상의 완전 연결(FC)층이 따라온다

#### 이산 합성곱

이산 합성곱(discrete convolution)이 CNN의 기본 연산이며, 다음은 1차원 합성곱의 정의하는 식이다

$$y = x \cdot w \rightarrow y[i] = \sum_{k = -\infty}^{+\infty} x[i-k]w[k]$$

여기서 $x$는 입력 벡터, $w$는 필터(filter) 또는 커널(kernel)벡터라고 한다

In [1]:
import numpy as np

def Con1D(x, w, p = 0, s = 1) :
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int(len(x)/s),s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print(Con1D(x, w, p = 2, s = 1))

[ 5. 14. 16. 26. 24. 34. 19. 22.]


In [3]:
print(np.convolve(x, w, mode = 'same')) # numpy

[ 5 14 16 26 24 34 19 22]


#### 2D 이산 합성곱

In [7]:
import scipy.signal

def Conv2D(X, W, p=(0,0), s=(1,1)):
    W_rot = np.array(W)[::-1,::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1,n2))
    X_padded[p[0]:p[0] + X_orig.shape[0], 
             p[1]:p[1] + X_orig.shape[1]] = X_orig

    res = []
    for i in range(0, int((X_padded.shape[0] - 
                           W_rot.shape[0])/s[0])+1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - 
                               W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

X = [
    [1, 3, 2, 4], 
    [5, 6, 1, 3], 
    [1 ,2, 0, 2], 
    [3, 4, 3, 2]
]
W = [
    [1, 0, 3], 
    [1, 2, 1],
    [0, 1, 1]
]

Conv2D(X, W, p = (1, 1), s = (1, 1))

array([[11., 25., 32., 13.],
       [19., 25., 24., 13.],
       [13., 28., 25., 17.],
       [11., 17., 14.,  9.]])

In [6]:
scipy.signal.convolve2d(X, W, mode = 'same')

array([[11, 25, 32, 13],
       [19, 25, 24, 13],
       [13, 28, 25, 17],
       [11, 17, 14,  9]])

**합성곱 출력의 크기 계산**

합성곱 출력 크기는 입력 벡터 위를 $w$가 이동하는 전체 횟수로 결정된다. 이전과 같이 입력 벡터의 크기가 $n$인 필터의 크기가 $m$, 패딩이 $p$, 스트라이드가 $s$인 $x \cdot w$의 출력 크기는 다음과 같이 계산된다

$$o = \Big[ \frac{n + 2p - m}{s} \Big] + 1$$

이전의 정의를 바탕으로 2차원 합성곱 역시 정의 가능하다. $m_1 \leq n_1$이고 $m_2 \leq n_2$인 입력 행렬 $X_{n_1 \times n_2}$와 필터 행렬 $W_{m_1 \times m_2}$의 합성곱 행렬 $Y = XW$는 다음과 같이 정의된다

Inner product of two vectors $X = (x_1, x_2, x_3, \dots, x_n)$ and $Y = (y_1, y_2, y_3, \dots, y_n)$

$$X \cdot Y = \sum_{i=1}^n x_i y_i \in \mathcal{R}$$

Inner product of two square matrices $A = (a_{ij})_{n \times n}, B = (b_{ij})_{n \times n}$

$$A \cdot B = AB^{T} \notin \mathcal{R}$$
$$= \mbox{trace}{AB^{T}} \in \mathcal{R}$$
$$= \begin{pmatrix}
 a_{11} & a_{12} & \dots & a_{1n} \\
 a_{21} & a_{22}  & \dots & a_{2n} \\
 \vdots & \vdots & \ddots & \vdots \\
 a_{n1} & a_{n2}  & \dots & a_{nn}
 \end{pmatrix}
 \begin{pmatrix}
 b_{11} & b_{12} & \dots & b_{1n} \\
 b_{21} & b_{22}  & \dots & b_{2n} \\
 \vdots & \vdots & \ddots & \vdots \\
 b_{n1} & b_{n2}  & \dots & b_{nn}
 \end{pmatrix}$$